In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

from ldm.data import image_processing
from ldm import util

In [3]:
metadata = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv", index_col=0)
metadata

/tmp/ipykernel_42171/397491088.py:1: DtypeWarning: Columns (8,9,15,17,19,20,34,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv", index_col=0)


,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,image_id,imputation_split,protcl_denovo_split,prot_denovo_split,ImageHeight,ImageWidth
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,7.0,NaN,1_A1_1,train,test,train,1728.0,1728.0
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,Supported,Supported,6.0,NaN,1_A1_2,train,test,train,1728.0,1728.0
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,5.0,NaN,1_A10_1,NaN,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,NaN,4.0,NaN,1_A10_2,NaN,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,NaN,3.0,NaN,1_A11_1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_20,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_21,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_22,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,Approved,NaN,NaN,NaN,2184_H8_23,NaN,NaN,NaN,NaN,NaN


In [25]:
def rescale_image(image_id):
    plate_id = image_id.split("_")[0]
    save_dir = "/scratch/groups/emmalu/HPA_rescaled"
    rescaled_image_path = f"{save_dir}/{image_id}.tif"
    if not os.path.exists(rescaled_image_path):
    # print(image_id)
    # if True:
        try:
            full_res_image = image_processing.load_full_res_image(plate_id, image_id)
            Image.fromarray(full_res_image).save(rescaled_image_path)
        except FileNotFoundError:
            pass

# rescale_image(metadata.loc[0, "image_id"])
with util.MyPool(processes=1, chunksize=1000) as p:
    list(tqdm(p.map_func(rescale_image, metadata["image_id"]), total=len(metadata)))

  2%|█▌                                                                                              | 17240/1086890 [00:28<29:25, 606.00it/s]


KeyboardInterrupt: 